![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_RoBertaForTokenClassification.ipynb)

# Import OpenVINO RoBertaForTokenClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting RoBertaForTokenClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for RoBertaForTokenClassification from RoBertaForTokenClassification   and they have to be in `Token Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
 

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [obi/deid_roberta_i2b2](https://huggingface.co/obi/deid_roberta_i2b2) model from HuggingFace as an example and load it as a `OVModelForTokenClassification`, representing an OpenVINO model.
- In addition to the OVModelForTokenClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForTokenClassification
from transformers import AutoTokenizer

MODEL_NAME = "obi/deid_roberta_i2b2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/obi/deid_roberta_i2b2/tokenizer_config.json',
 'ov_models/obi/deid_roberta_i2b2/special_tokens_map.json',
 'ov_models/obi/deid_roberta_i2b2/vocab.json',
 'ov_models/obi/deid_roberta_i2b2/merges.txt',
 'ov_models/obi/deid_roberta_i2b2/added_tokens.json',
 'ov_models/obi/deid_roberta_i2b2/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {EXPORT_PATH}

total 1389640
-rw-r--r-- 1 root root       2505 Jun 24 12:13 config.json
-rw-r--r-- 1 root root     456318 Jun 24 12:13 merges.txt
-rw-r--r-- 1 root root 1417429348 Jun 24 12:13 openvino_model.bin
-rw-r--r-- 1 root root     724181 Jun 24 12:13 openvino_model.xml
-rw-r--r-- 1 root root        964 Jun 24 12:13 special_tokens_map.json
-rw-r--r-- 1 root root       1383 Jun 24 12:13 tokenizer_config.json
-rw-r--r-- 1 root root    3558743 Jun 24 12:13 tokenizer.json
-rw-r--r-- 1 root root     798293 Jun 24 12:13 vocab.json


- We need to convert `vocab.json` to a plain `vocab.txt` format, as required by Spark NLP.
- Move both `vocab.txt` and `merges.txt` into the assets folder.
- Additionally, we need to extract label-to-ID mappings from the model config and save them as `labels.txt` in the same folder for Spark NLP to use during inference.

In [6]:
import json

!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

with open(f"{EXPORT_PATH}/vocab.json") as f, open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as out:
    out.write("\n".join(json.load(f)))

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(ov_model.config.id2label[k] for k in sorted(ov_model.config.id2label)))

In [7]:
!ls -lR {EXPORT_PATH}

ov_models/obi/deid_roberta_i2b2:
total 1389196
drwxr-xr-x 2 root root       4096 Jun 24 12:15 assets
-rw-r--r-- 1 root root       2505 Jun 24 12:13 config.json
-rw-r--r-- 1 root root 1417429348 Jun 24 12:13 openvino_model.bin
-rw-r--r-- 1 root root     724181 Jun 24 12:13 openvino_model.xml
-rw-r--r-- 1 root root        964 Jun 24 12:13 special_tokens_map.json
-rw-r--r-- 1 root root       1383 Jun 24 12:13 tokenizer_config.json
-rw-r--r-- 1 root root    3558743 Jun 24 12:13 tokenizer.json
-rw-r--r-- 1 root root     798293 Jun 24 12:13 vocab.json

ov_models/obi/deid_roberta_i2b2/assets:
total 852
-rw-r--r-- 1 root root    341 Jun 24 12:15 labels.txt
-rw-r--r-- 1 root root 456318 Jun 24 12:13 merges.txt
-rw-r--r-- 1 root root 407064 Jun 24 12:15 vocab.txt


Voila! We have our `vocab.txt` and `merges.txt` inside assets directory, along with the extracted labels saved in `labels.txt`.

## Import and Save RoBertaForTokenClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [3]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 30.1 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [4]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [8]:
from sparknlp.annotator import RoBertaForTokenClassification

tokenClassifier = RoBertaForTokenClassification\
  .loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your RoBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 1385140
drwxr-xr-x 5 root root       4096 Jun 24 12:16 fields
drwxr-xr-x 2 root root       4096 Jun 24 12:16 metadata
-rw-r--r-- 1 root root 1418370207 Jun 24 12:17 roberta_classification_openvino


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForTokenClassification model 😊

In [12]:
tokenClassifier_loaded = RoBertaForTokenClassification.load("./{}_spark_nlp_openvino".format(EXPORT_PATH))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function: introduced in `spark-nlp==3.4.0`

In [13]:
tokenClassifier_loaded.getClasses()

['L-HOSP',
 'U-LOC',
 'I-AGE',
 'B-STAFF',
 'U-AGE',
 'B-LOC',
 'L-AGE',
 'B-DATE',
 'L-OTHERPHI',
 'U-ID',
 'U-PATORG',
 'I-DATE',
 'U-OTHERPHI',
 'B-PHONE',
 'I-HOSP',
 'U-EMAIL',
 'L-ID',
 'L-PATIENT',
 'L-PATORG',
 'L-STAFF',
 'U-PHONE',
 'I-PHONE',
 'I-EMAIL',
 'B-AGE',
 'B-ID',
 'L-LOC',
 'B-PATORG',
 'B-EMAIL',
 'L-EMAIL',
 'U-DATE',
 'I-OTHERPHI',
 'I-LOC',
 'B-OTHERPHI',
 'I-STAFF',
 'I-PATIENT',
 'U-STAFF',
 'I-PATORG',
 'L-PHONE',
 'U-PATIENT',
 'U-HOSP',
 'B-HOSP',
 'B-PATIENT',
 'O',
 'I-ID',
 'L-DATE']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [14]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

ner_converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    ner_converter
])

example_data = spark.createDataFrame([
    ["Patient John Doe was admitted to New York Hospital on 12/05/2022 for chest pain."],
    ["Dr. Emily Smith prescribed Aspirin to the patient at Boston Medical Center on 03-15-2023."]
], ["text"])

model = pipeline.fit(example_data)
result = model.transform(example_data)

result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+------------------------+-------+
|text                    |entity |
+------------------------+-------+
|John Doe                |PATIENT|
|New York Hospital       |HOSP   |
|Emily Smith             |STAFF  |
|Boston Medical Center on|HOSP   |
+------------------------+-------+



That's it! You can now go wild and use hundreds of `RoBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
